In [1]:
import evoVAE.utils.seq_tools as st
import evoVAE.utils.statistics as stats
import pandas as pd


pd.set_option("display.max_rows", None)

In [7]:
test_aln = st.read_aln_file("../data/pair_test.aln")
test_aln


Reading the alignment: ../data/pair_test.aln
Checking for bad characters: ['B', 'J', 'X', 'Z']
Performing one hot encoding
Number of seqs: 8


,id,sequence,encoding
0,seq_1,LTRAALYEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,seq_2,ATRATLYEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seq_3,TTRCTLPEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
3,seq_4,WRRATLPDDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,seq_5,KRRATLPDDA,"[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,seq_6,PVRATKPWDA,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,seq_7,DVRATLPWDA,"[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
7,seq_8,PVRATLPWDA,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [8]:
pfm = stats.calc_position_freq_matrix(test_aln)
pfm[:, 0]

Sequence converted to numpy array with shape (8, 10)


array([0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 2., 1., 0., 0., 1.,
       0., 0., 0., 1.])

In [9]:
ppm = stats.calc_position_prob_matrix(test_aln)
ppm[:, 0]

Sequence converted to numpy array with shape (8, 10)


array([0.   , 0.   , 0.   , 0.125, 0.125, 0.   , 0.   , 0.125, 0.   ,
       0.   , 0.   , 0.   , 0.25 , 0.125, 0.   , 0.   , 0.125, 0.   ,
       0.   , 0.   , 0.125])

In [10]:
H = stats.calc_shannon_entropy(test_aln)
H



Sequence converted to numpy array with shape (8, 10)


array([ 1.90615475,  1.08219553, -0.        ,  0.37677016,  0.37677016,
        0.37677016,  0.56233514,  1.08219553, -0.        ,  0.69314718])